In [3]:
from stanfordcorenlp import StanfordCoreNLP

In [17]:
s = '12月16日电'

In [9]:
nlp = StanfordCoreNLP('/home/xy/stanford-corenlp-full-2018-02-27/', lang='zh')

In [18]:
nlp.word_tokenize(s)

['12月', '16日', '电']